In [1]:
pwd

'C:\\Users\\Diju\\Downloads'

In [2]:
!pip install keras
!pip install tensorflow

In [3]:
import keras
keras.__version__

'2.11.0'

In [4]:
import tensorflow
tensorflow.__version__

'2.11.0'

Importing Neccessary Libraries

In [5]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

Image Data Agumentation

In [6]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [7]:
test_datagen=ImageDataGenerator(rescale=1./255)

Loading our data and performing data agumentation

In [10]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='w05F-ExgHa92z8IFYDU2wGoCOIj2nLxfRQgMxFzVNSkE',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'cnnclassification-donotdelete-pr-3zi8nzgqgzgeg5'
object_key = 'Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


NameError: name 'ibm_boto3' is not defined

In [9]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
print(file_paths)
for path in file_paths:
    unzip.extract(path)

NameError: name 'streaming_body_1' is not defined

In [13]:
import os
filenames = os.listdir('/home/wsuser/work/data')

In [14]:
x_train=train_datagen.flow_from_directory('/home/wsuser/work/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')
#performing data agumentation to test data
x_test=test_datagen.flow_from_directory('/home/wsuser/work/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [15]:
print(x_train.class_indices)

{'Left Bundle Branch Block': 0, 'Normal': 1, 'Premature Atrial Contraction': 2, 'Premature Ventricular Contractions': 3, 'Right Bundle Branch Block': 4, 'Ventricular Fibrillation': 5}


In [16]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 504, 1: 7346, 2: 2054, 3: 2759, 4: 2239, 5: 439})

Creating the model

In [17]:
# create model
model=Sequential()
# adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))#convolutional layer
model.add(MaxPooling2D(pool_size=(2,2))) #MaxPooling2D-for downsampling the input

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(32))#deeply connected neural network layers.
model.add(Dense(6,activation='softmax'))#output layer with 6 neurons

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

Compiling the model

In [19]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Fitting the model

In [20]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),epochs=10, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_165/2406469344.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),epochs=10, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/10
480/480 [==============================] - 67s 138ms/step - loss: 0.7253 - accuracy: 0.7565 - val_loss: 0.5020 - val_accuracy: 0.8519
Epoch 2/10
480/480 [==============================] - 67s 139ms/step - loss: 0.2856 - accuracy: 0.9144 - val_loss: 0.4196 - val_accuracy: 0.8749
Epoch 3/10
480/480 [==============================] - 65s 136ms/step - loss: 0.2324 - accuracy: 0.9328 - val_loss: 0.3256 - val_accuracy: 0.9140
Epoch 4/10
480/480 [==============================] - 66s 138ms/step - loss: 0.1876 - accuracy: 0.9438 - val_loss: 0.3377 - val_accuracy: 0.8949
Epoch 5/10
480/480 [==============================] - 66s 137ms/step - loss: 0.1699 - accuracy: 0.9489 - val_loss: 0.3558 - val_accuracy: 0.8838
Epoch 6/10
480/480 [==============================] - 68s 141ms/step - loss: 0.1470 - accuracy: 0.9558 - val_loss: 0.3346 - val_accuracy: 0.8999
Epoch 7/10
480/480 [==============================] - 71s 148ms/step - loss: 0.1363 - accuracy: 0.9591 - val_loss: 0.2873 - val_ac

Saving our model

In [21]:
from tensorflow.keras.models import load_model
model.save('ECG.h5')

In [22]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 15.5 MB/s eta 0:00:01


In [23]:
!tar -zcvf ECG-classification-model_new.tgz ECG.h5

ECG.h5


In [24]:
ls 

data/  ECG-classification-model_new.tgz  ECG.h5


In [114]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"dbaRZleBA_vVlq0ubIczAPD4W2Vtx6JB39Sj3kLeyXFJ"
                  }
client = APIClient(wml_credentials)

In [115]:
client = APIClient(wml_credentials)

In [116]:
client

In [117]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [118]:
space_uid = guid_from_space_name(client, 'ecg_verification')
print("Space UID = "+ space_uid)

Space UID = cd8bd310-bc2b-4310-96fa-4278b2d69f70


In [119]:
client.set.default_space(space_uid)

'SUCCESS'

In [120]:
client.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [121]:
import tensorflow
tensorflow.__version__

'2.7.2'

In [126]:
software_space_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")


In [127]:
software_space_uid
    

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [124]:
!tar -zcvf ECG-classification-model_new.tgz ECG.h5

ECG.h5


In [130]:
model_details=client.repository.store_model(model='ECG-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"cnn_classification",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid})

model_id=client.repository.get_model_id(model_details)

                                           

In [131]:
model_id

'ecc413a8-f596-42db-a091-cb9b42d2f974'

In [132]:
client.repository.download(model_id, 'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [133]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [134]:
model = load_model("ECG.h5")